In [1]:
import bs4 as BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
main_url = 'https://www.basketball-reference.com/players/'
letters = 'abcdefghijklmnopqrstuvwxyz'
letters_list = list(letters)

In [2]:
""" htmldata = urllib.request.urlopen('https://www.basketball-reference.com/players/a/')
bsoup = BeautifulSoup.BeautifulSoup(htmldata,'html.parser')
complete_array = None
Data_list = None """

for letter in letters_list:
    print(letter)
    try:
        htmldata = urllib.request.urlopen('https://www.basketball-reference.com/players/' + letter +'/')
    except:
        continue
    bsoup = BeautifulSoup.BeautifulSoup(htmldata,'html.parser')
    main_array = None
    complete_array = None
    Data_list = None
    for i in range(1,len(bsoup.find_all('tr'))):
        #we collect only players who played after 1980
        if int(bsoup.find_all('tr')[i].find('td',{'data-stat':'year_max'}).get_text())>1980:
            href = bsoup.find_all('tr')[i].find_all('a')[0].attrs['href']
            try:
                player_html = urllib.request.urlopen(main_url + letter + "/" + href.split('/')[-1])
            except:
                #due to errors in the html page, some href do not work hence we skip these
                continue
            player_bsoup = BeautifulSoup.BeautifulSoup(player_html,'html.parser')
            if Data_list == None:
                Data_list = [item.get_text() for item in player_bsoup.find_all('th') if 'aria-label' in item.attrs.keys()]
                Data_list.insert(0,'Name')
            player_dataarray = np.empty(shape = (len(player_bsoup.find_all('tbody')[0].find_all('tr')),len(Data_list)), dtype = np.object)
            for j in range(0,len(player_bsoup.find_all('tbody')[0].find_all('tr'))):
                data_row = [player_bsoup.find_all('h1',{'itemprop':'name'})[0].get_text().strip()]
                #getting season
                try:
                    #some players wont play some years, at those instances, this code will throw exception
                    #hence we continue to the next year
                    data_row.extend([player_bsoup.find_all('tbody')[0].find_all('tr')[j].find_all('th')[0].get_text()])
                    #adding rest of data    
                    [data_row.append(data) for data in [td.get_text() if td.get_text() != "" else np.nan for td in (player_bsoup.find_all                     ('tbody')[0].find_all('tr')[j].find_all('td'))]]    
                except:
                    continue
                try:     
                    player_dataarray[j,:]  = np.array(data_row)
                except:
                    #any player who doesnt have ['Name','Season','Age','Tm','Lg','Pos','G','GS','MP','FG','FGA','FG%','3P','3PA','3P%', '2P''2PA','2P%','eFG%','FT','FTA','FT%','ORB','DRB','TRB','AST','STL','BLK','TOV','PF','PTS'] as there stats are not considered. Majority of players after 1980 will have these stats as these were collected and recorded in this era of basketball
                    break
        if i == 1:
            complete_array = player_dataarray
        else:
            complete_array = np.vstack((complete_array,player_dataarray))
    if letter == a:
        main_array = complete_array
        print(main_array.shape)
    else:
        main_array = np.vstack((main_array,complete_array))
        print(main_array.shape)
        
    


In [6]:
complete_array.shape

(97, 31)

array([False, False, False, ..., False, False, False])

In [55]:
list(player_bsoup.find_all('tr'))[1]

<tr class="full_table" id="per_game.1991"><th class="left" data-stat="season" scope="row"><a href="/players/a/abdelal01/gamelog/1991/">1990-91</a></th><td class="center" data-stat="age">22</td><td class="left" data-stat="team_id"><a href="/teams/POR/1991.html">POR</a></td><td class="left" data-stat="lg_id"><a href="/leagues/NBA_1991.html">NBA</a></td><td class="center" data-stat="pos">PF</td><td class="right" data-stat="g">43</td><td class="right iz" data-stat="gs">0</td><td class="right" data-stat="mp_per_g">6.7</td><td class="right" data-stat="fg_per_g">1.3</td><td class="right" data-stat="fga_per_g">2.7</td><td class="right" data-stat="fg_pct">.474</td><td class="right iz" data-stat="fg3_per_g">0.0</td><td class="right iz" data-stat="fg3a_per_g">0.0</td><td class="right iz" data-stat="fg3_pct"></td><td class="right" data-stat="fg2_per_g">1.3</td><td class="right" data-stat="fg2a_per_g">2.7</td><td class="right" data-stat="fg2_pct">.474</td><td class="right" data-stat="efg_pct">.474<

In [57]:
list(player_bsoup.find_all('tr'))[2]

<tr class="full_table" id="per_game.1992"><th class="left" data-stat="season" scope="row"><a href="/players/a/abdelal01/gamelog/1992/">1991-92</a></th><td class="center" data-stat="age">23</td><td class="left" data-stat="team_id"><a href="/teams/POR/1992.html">POR</a></td><td class="left" data-stat="lg_id"><a href="/leagues/NBA_1992.html">NBA</a></td><td class="center" data-stat="pos">PF</td><td class="right" data-stat="g">71</td><td class="right" data-stat="gs">1</td><td class="right" data-stat="mp_per_g">13.2</td><td class="right" data-stat="fg_per_g">2.5</td><td class="right" data-stat="fga_per_g">5.1</td><td class="right" data-stat="fg_pct">.493</td><td class="right iz" data-stat="fg3_per_g">0.0</td><td class="right iz" data-stat="fg3a_per_g">0.0</td><td class="right iz" data-stat="fg3_pct"></td><td class="right" data-stat="fg2_per_g">2.5</td><td class="right" data-stat="fg2a_per_g">5.1</td><td class="right" data-stat="fg2_pct">.493</td><td class="right" data-stat="efg_pct">.493</t

In [56]:
list(player_bsoup.find_all('tr'))[-1]

<tr><th class="left" data-stat="season" scope="row">1 season</th><td class="center iz" data-stat="age"></td><td class="left" data-stat="team_id"><a href="/teams/PHI/">PHI</a></td><td class="left" data-stat="lg_id">NBA</td><td class="center iz" data-stat="pos"></td><td class="right" data-stat="g">3</td><td class="right iz" data-stat="gs">0</td><td class="right" data-stat="mp_per_g">10.0</td><td class="right" data-stat="fg_per_g">0.3</td><td class="right" data-stat="fga_per_g">3.7</td><td class="right" data-stat="fg_pct">.091</td><td class="right iz" data-stat="fg3_per_g">0.0</td><td class="right iz" data-stat="fg3a_per_g">0.0</td><td class="right iz" data-stat="fg3_pct"></td><td class="right" data-stat="fg2_per_g">0.3</td><td class="right" data-stat="fg2a_per_g">3.7</td><td class="right" data-stat="fg2_pct">.091</td><td class="right" data-stat="efg_pct">.091</td><td class="right iz" data-stat="ft_per_g">0.0</td><td class="right iz" data-stat="fta_per_g">0.0</td><td class="right iz" data

In [60]:
player_bsoup.find_all('tr')[-1]

<tr class="full_table" id="per_game.1995"><th class="left" data-stat="season" scope="row"><a href="/players/a/abdelal01/gamelog/1995/">1994-95</a></th><td class="center" data-stat="age">26</td><td class="left" data-stat="team_id">TOT</td><td class="left" data-stat="lg_id"><a href="/leagues/NBA_1995.html">NBA</a></td><td class="center" data-stat="pos">PF</td><td class="right" data-stat="g">54</td><td class="right iz" data-stat="gs">0</td><td class="right" data-stat="mp_per_g">9.4</td><td class="right" data-stat="fg_per_g">2.2</td><td class="right" data-stat="fga_per_g">4.3</td><td class="right" data-stat="fg_pct">.511</td><td class="right iz" data-stat="fg3_per_g">0.0</td><td class="right iz" data-stat="fg3a_per_g">0.0</td><td class="right iz" data-stat="fg3_pct">.000</td><td class="right" data-stat="fg2_per_g">2.2</td><td class="right" data-stat="fg2a_per_g">4.2</td><td class="right" data-stat="fg2_pct">.515</td><td class="right" data-stat="efg_pct">.511</td><td class="right" data-stat

In [65]:
player_bsoup.find_all('tbody')[0].find_all('tr')

[<tr class="full_table" id="per_game.1991"><th class="left" data-stat="season" scope="row"><a href="/players/a/abdelal01/gamelog/1991/">1990-91</a></th><td class="center" data-stat="age">22</td><td class="left" data-stat="team_id"><a href="/teams/POR/1991.html">POR</a></td><td class="left" data-stat="lg_id"><a href="/leagues/NBA_1991.html">NBA</a></td><td class="center" data-stat="pos">PF</td><td class="right" data-stat="g">43</td><td class="right iz" data-stat="gs">0</td><td class="right" data-stat="mp_per_g">6.7</td><td class="right" data-stat="fg_per_g">1.3</td><td class="right" data-stat="fga_per_g">2.7</td><td class="right" data-stat="fg_pct">.474</td><td class="right iz" data-stat="fg3_per_g">0.0</td><td class="right iz" data-stat="fg3a_per_g">0.0</td><td class="right iz" data-stat="fg3_pct"></td><td class="right" data-stat="fg2_per_g">1.3</td><td class="right" data-stat="fg2a_per_g">2.7</td><td class="right" data-stat="fg2_pct">.474</td><td class="right" data-stat="efg_pct">.474

In [92]:
player_bsoup.find_all('tbody')[0].find_all('tr')[0].find_all('td')

[<td class="center" data-stat="age">22</td>,
 <td class="left" data-stat="team_id"><a href="/teams/POR/1991.html">POR</a></td>,
 <td class="left" data-stat="lg_id"><a href="/leagues/NBA_1991.html">NBA</a></td>,
 <td class="center" data-stat="pos">PF</td>,
 <td class="right" data-stat="g">43</td>,
 <td class="right iz" data-stat="gs">0</td>,
 <td class="right" data-stat="mp_per_g">6.7</td>,
 <td class="right" data-stat="fg_per_g">1.3</td>,
 <td class="right" data-stat="fga_per_g">2.7</td>,
 <td class="right" data-stat="fg_pct">.474</td>,
 <td class="right iz" data-stat="fg3_per_g">0.0</td>,
 <td class="right iz" data-stat="fg3a_per_g">0.0</td>,
 <td class="right iz" data-stat="fg3_pct"></td>,
 <td class="right" data-stat="fg2_per_g">1.3</td>,
 <td class="right" data-stat="fg2a_per_g">2.7</td>,
 <td class="right" data-stat="fg2_pct">.474</td>,
 <td class="right" data-stat="efg_pct">.474</td>,
 <td class="right" data-stat="ft_per_g">0.6</td>,
 <td class="right" data-stat="fta_per_g">1.0</

In [82]:
'aria-label' in list(player_bsoup.find_all('th')[0].attrs.keys())

True

In [88]:
Data_list = [item.get_text() for item in player_bsoup.find_all('th') if 'aria-label' in item.attrs.keys()]

In [89]:
Data_list.insert(0,'Name')
Data_list

['Name',
 'Season',
 'Age',
 'Tm',
 'Lg',
 'Pos',
 'G',
 'GS',
 'MP',
 'FG',
 'FGA',
 'FG%',
 '3P',
 '3PA',
 '3P%',
 '2P',
 '2PA',
 '2P%',
 'eFG%',
 'FT',
 'FTA',
 'FT%',
 'ORB',
 'DRB',
 'TRB',
 'AST',
 'STL',
 'BLK',
 'TOV',
 'PF',
 'PTS']

In [153]:
import numpy as np
#getting player name
Data_list = [item.get_text() for item in player_bsoup.find_all('th') if 'aria-label' in item.attrs.keys()]
Data_list.insert(0,'Name')
player_dataarray = np.empty(shape = (len(player_bsoup.find_all('tbody')[0].find_all('tr')),len(Data_list)), dtype = np.object)
for i in range(0,len(player_bsoup.find_all('tbody')[0].find_all('tr'))):
    data_row = [player_bsoup.find_all('h1',{'itemprop':'name'})[0].get_text().strip()]
    #getting season
    data_row.extend([player_bsoup.find_all('tbody')[0].find_all('tr')[i].find_all('th')[0].get_text()])
    #adding rest of data
    [data_row.append(data) for data in [td.get_text() if td.get_text() != "" else np.nan for td in (player_bsoup.find_all('tbody')[0].find_all('tr')[i].find_all('td'))]]         
    player_dataarray[i,:]  = np.array(data_row)




In [154]:
player_dataarray

array([['Alaa Abdelnaby', '1990-91', '22', 'POR', 'NBA', 'PF', '43', '0',
        '6.7', '1.3', '2.7', '.474', '0.0', '0.0', 'nan', '1.3', '2.7',
        '.474', '.474', '0.6', '1.0', '.568', '0.6', '1.4', '2.1', '0.3',
        '0.1', '0.3', '0.5', '0.9', '3.1'],
       ['Alaa Abdelnaby', '1991-92', '23', 'POR', 'NBA', 'PF', '71', '1',
        '13.2', '2.5', '5.1', '.493', '0.0', '0.0', 'nan', '2.5', '5.1',
        '.493', '.493', '1.1', '1.4', '.752', '1.1', '2.5', '3.7', '0.4',
        '0.4', '0.2', '0.9', '1.9', '6.1'],
       ['Alaa Abdelnaby', '1992-93', '24', 'TOT', 'NBA', 'PF', '75',
        '52', '17.5', '3.3', '6.3', '.518', '0.0', '0.0', '.000', '3.3',
        '6.3', '.519', '.518', '1.2', '1.5', '.759', '1.7', '2.8', '4.5',
        '0.4', '0.3', '0.3', '1.3', '2.5', '7.7'],
       ['Alaa Abdelnaby', '1992-93', '24', 'MIL', 'NBA', 'PF', '12', '0',
        '13.3', '2.2', '4.7', '.464', '0.0', '0.1', '.000', '2.2', '4.6',
        '.473', '.464', '1.0', '1.3', '.750', '1.0', '2.

In [150]:
import numpy as np

In [168]:
x = np.array([[1],[2],[3]])
y = np.array([4,5,6])

In [175]:
np.c_[x,y.reshape(3,1)]

array([[1, 4],
       [2, 5],
       [3, 6]])